In [10]:

import os
import random
import time

class WumpusWorld:
    def __init__(self, size=4):
        self.size = size
        self.environment = [[' ' for _ in range(size)] for _ in range(size)]
        self.player_pos = [size-1, 0]  # Start at the bottom-left corner
        self.wumpus_pos = self.get_unique_position(exclude=[self.player_pos])
        self.pit_pos = self.get_unique_position(exclude=[self.player_pos, self.wumpus_pos])
        self.gold_pos = self.get_unique_position(exclude=[self.player_pos, self.wumpus_pos, self.pit_pos])
        self.wumpus_alive = True  # Track if Wumpus is alive
        self.visited = set()
        self.safe_zones = set()
        self.unsafe_zones = set()
        self.moves = 0
        self.has_arrow = True  # Player starts with one arrow
        self.agent_direction = 'right'  # The initial direction the agent is facing
        self.bump_percept = False
        self.scream_percept = False

    def get_unique_position(self, exclude):
        """Find a unique position in the grid that is not in the exclude list."""
        while True:
            pos = [random.randint(0, self.size-1), random.randint(0, self.size-1)]
            if pos not in exclude:
                return pos

    def clear_screen(self):
        os.system('cls' if os.name == 'nt' else 'clear')

    def print_environment(self):
        self.clear_screen()
        print("Wumpus World:")
        print("-------------")
        for i in range(self.size):
            row = ""
            for j in range(self.size):
                cell_content = ""
                if [i, j] == self.player_pos:
                    cell_content += "A"  # Agent
                else:
                    percepts = self.get_percepts_at([i, j])
                    if percepts["glitter"]:
                        cell_content += "G"  # Glitter (Gold)
                    elif [i, j] == self.wumpus_pos and self.wumpus_alive:
                        cell_content += "W"  # Wumpus
                    elif [i, j] == self.pit_pos:
                        cell_content += "P"  # Pit
                    else:
                        if percepts["stench"] and self.wumpus_alive:
                            cell_content += "S"  # Stench
                        if percepts["breeze"]:
                            cell_content += "B"  # Breeze
                        if not cell_content:
                            cell_content = "."

                row += f" {cell_content} "
            print(row)
        print("-------------\n")

    def get_percepts_at(self, position):
        percepts = {"breeze": False, "stench": False, "glitter": False}
        x, y = position

        if position == self.gold_pos:
            percepts["glitter"] = True

        if self.wumpus_alive and any([x, y] == [self.wumpus_pos[0] + dx, self.wumpus_pos[1] + dy] for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]):
            percepts["stench"] = True

        if any([x, y] == [self.pit_pos[0] + dx, self.pit_pos[1] + dy] for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]):
            percepts["breeze"] = True

        return percepts

    def get_percepts(self):
        return self.get_percepts_at(self.player_pos)

    def move_player(self, direction):
        x, y = self.player_pos
        self.agent_direction = direction  # Update the agent's facing direction

        if direction == 'up' and x > 0:
            self.player_pos = [x-1, y]
            self.bump_percept = False
        elif direction == 'down' and x < self.size-1:
            self.player_pos = [x+1, y]
            self.bump_percept = False
        elif direction == 'left' and y > 0:
            self.player_pos = [x, y-1]
            self.bump_percept = False
        elif direction == 'right' and y < self.size-1:
            self.player_pos = [x, y+1]
            self.bump_percept = False
        else:
            print("Bump! You hit the boundary.")
            self.bump_percept = True  # If the agent tries to move beyond the boundary
            return False

        self.moves += 1
        return True

    def update_knowledge_base(self):
        self.visited.add(tuple(self.player_pos))
        percepts = self.get_percepts()
        
        if not percepts["breeze"] and not percepts["stench"]:
            self.safe_zones.update(self.get_adjacent_cells(self.player_pos))
        
        if percepts["breeze"]:
            self.unsafe_zones.update(self.get_adjacent_cells(self.player_pos))
        
        if percepts["stench"]:
            self.unsafe_zones.update(self.get_adjacent_cells(self.player_pos))

    def get_adjacent_cells(self, position):
        x, y = position
        adjacent_cells = []
        if x > 0:
            adjacent_cells.append((x-1, y))
        if x < self.size - 1:
            adjacent_cells.append((x+1, y))
        if y > 0:
            adjacent_cells.append((x, y-1))
        if y < self.size - 1:
            adjacent_cells.append((x, y+1))
        return adjacent_cells

    def choose_next_move(self):
        # Get the current percepts
        percepts = self.get_percepts()

        # If there's a scream, the Wumpus is dead, and we can ignore stenches
        if self.scream_percept:
            print("Wumpus is dead! No need to worry about stenches.")
            self.scream_percept = False  # Reset scream percept after processing

        # If there's a stench and the Wumpus is still alive, consider shooting the arrow
        if percepts["stench"] and self.wumpus_alive:
            wumpus_inferred_pos = self.infer_wumpus_location()
            if wumpus_inferred_pos:
                # Calculate direction to shoot the arrow
                direction_to_shoot = self.get_direction_to(wumpus_inferred_pos)
                if direction_to_shoot == self.agent_direction:
                    print("AI decides to shoot the arrow!")
                    self.shoot_arrow()
                    return None
                else:
                    print(f"AI decides to turn towards {direction_to_shoot} to shoot.")
                    return direction_to_shoot  # Rotate to face the correct direction for shooting

        # Check if the bump percept was triggered on the last move
        if self.bump_percept:
            print("Bump! AI hit a boundary.")
            self.bump_percept = False  # Reset bump percept after processing
            # AI should avoid moving in the same direction that caused the bump
            return self.choose_direction_to_avoid_boundary()

        # Otherwise, move to the next safe zone
        for cell in self.safe_zones:
            if cell not in self.visited:
                x, y = cell
                px, py = self.player_pos
                if x < px:
                    return "up"
                elif x > px:
                    return "down"
                elif y < py:
                    return "left"
                elif y > py:
                    return "right"

        # If no safe moves are available, take a random valid move
        print("No safe moves available. AI will take a random move.")
        return self.choose_random_valid_move()

    def choose_random_valid_move(self):
        """Choose a random valid move when no safe moves are available."""
        x, y = self.player_pos
        valid_directions = []

        if x > 0:
            valid_directions.append("up")
        if x < self.size - 1:
            valid_directions.append("down")
        if y > 0:
            valid_directions.append("left")
        if y < self.size - 1:
            valid_directions.append("right")

        # Randomly choose a valid direction
        if valid_directions:
            return random.choice(valid_directions)
        else:
            return None  # If somehow no valid directions, return None

    def infer_wumpus_location(self):
        """Infer the most probable location of the Wumpus based on stench and visited cells."""
        adjacent_cells = self.get_adjacent_cells(self.player_pos)
        possible_wumpus_cells = [cell for cell in adjacent_cells if cell not in self.visited]
        
        # If multiple adjacent cells have stench, we can narrow down Wumpus's location
        for cell in possible_wumpus_cells:
            x, y = cell
            if (x, y) in self.unsafe_zones and (x, y) != self.pit_pos:
                return (x, y)  # Return the inferred position of the Wumpus
        
        return None

    def get_direction_to(self, target_pos):
        """Determine the direction to face to shoot towards the target position."""
        px, py = self.player_pos
        tx, ty = target_pos

        if tx < px:
            return "up"
        elif tx > px:
            return "down"
        elif ty < py:
            return "left"
        elif ty > py:
            return "right"

    def shoot_arrow(self):
        """Automatically shoot the arrow in the direction the agent is facing."""
        if not self.has_arrow:
            print("You have no arrows left!")
            return False

        self.has_arrow = False  # Arrow can only be shot once
        px, py = self.player_pos

        # Shoot based on agent's current facing direction
        if self.agent_direction == 'up':
            return self.shoot_in_direction("up")
        elif self.agent_direction == 'down':
            return self.shoot_in_direction("down")
        elif self.agent_direction == 'left':
            return self.shoot_in_direction("left")
        elif self.agent_direction == 'right':
            return self.shoot_in_direction("right")

    def shoot_in_direction(self, direction):
        """Handles shooting the arrow in a specified direction."""
        px, py = self.player_pos
        wx, wy = self.wumpus_pos

        if direction == "up":
            for i in range(px - 1, -1, -1):  # Move upwards
                if (i, py) == (wx, wy):
                    self.wumpus_alive = False
                    self.scream_percept = True  # Set scream percept when Wumpus is killed
                    print("You shot the Wumpus! Scream!")
                    self.safe_zones.add((wx, wy))  # Mark the Wumpus's position as safe
                    return True
                self.safe_zones.add((i, py))  # Mark the path as safe
        elif direction == "down":
            for i in range(px + 1, self.size):  # Move downwards
                if (i, py) == (wx, wy):
                    self.wumpus_alive = False
                    self.scream_percept = True  # Set scream percept when Wumpus is killed
                    print("You shot the Wumpus! Scream!")
                    self.safe_zones.add((wx, wy))
                    return True
                self.safe_zones.add((i, py))
        elif direction == "left":
            for j in range(py - 1, -1, -1):  # Move left
                if (px, j) == (wx, wy):
                    self.wumpus_alive = False
                    self.scream_percept = True  # Set scream percept when Wumpus is killed
                    print("You shot the Wumpus! Scream!")
                    self.safe_zones.add((wx, wy))
                    return True
                self.safe_zones.add((px, j))
        elif direction == "right":
            for j in range(py + 1, self.size):  # Move right
                if (px, j) == (wx, wy):
                    self.wumpus_alive = False
                    self.scream_percept = True  # Set scream percept when Wumpus is killed
                    print("You shot the Wumpus! Scream!")
                    self.safe_zones.add((wx, wy))
                    return True
                self.safe_zones.add((px, j))

        print("Your arrow missed! The path is safe.")
        return False

    def check_status(self):
        if self.player_pos == self.wumpus_pos and self.wumpus_alive:
            print("Player encountered the Wumpus. Game Over!")
            return False
        elif self.player_pos == self.pit_pos:
            print("Player fell into a pit. Game Over!")
            return False
        elif self.player_pos == self.gold_pos:
            print("Player found the gold. You Win!")
            return False
        return True

def play_game():
    wumpus_world = WumpusWorld()
    print("Welcome to the Wumpus World!")
    num_moves = int(input("Enter the number of moves: "))

    while wumpus_world.moves < num_moves:
        wumpus_world.print_environment()
        wumpus_world.update_knowledge_base()

        move = wumpus_world.choose_next_move()
        if move is None:
            print("No safe moves available. Game Over!")
            break

        print(f"AI decided to move {move}.")
        if not wumpus_world.move_player(move):
            continue

        if not wumpus_world.check_status():
            break

        # Automatically shoot Wumpus if stench is detected in current cell
        if wumpus_world.get_percepts()["stench"] and wumpus_world.wumpus_alive:
            wumpus_world.shoot_arrow()

        time.sleep(1)  # Pause to simulate movement and give a clearer step-by-step display

    if wumpus_world.moves == num_moves:
        print("Out of moves! Game Over.")

if __name__ == "__main__":
    play_game()


Welcome to the Wumpus World!


Enter the number of moves:  8


Wumpus World:
-------------
 G  SB  W  S 
 B  P  SB  . 
 .  B  .  . 
 A  .  .  . 
-------------

AI decided to move right.
Wumpus World:
-------------
 G  SB  W  S 
 B  P  SB  . 
 .  B  .  . 
 .  A  .  . 
-------------

AI decided to move up.
Wumpus World:
-------------
 G  SB  W  S 
 B  P  SB  . 
 .  A  .  . 
 .  .  .  . 
-------------

AI decided to move left.
Wumpus World:
-------------
 G  SB  W  S 
 B  P  SB  . 
 A  B  .  . 
 .  .  .  . 
-------------

AI decided to move up.
Wumpus World:
-------------
 G  SB  W  S 
 A  P  SB  . 
 .  B  .  . 
 .  .  .  . 
-------------

AI decided to move down.
Wumpus World:
-------------
 G  SB  W  S 
 B  P  SB  . 
 A  B  .  . 
 .  .  .  . 
-------------

AI decided to move down.
Wumpus World:
-------------
 G  SB  W  S 
 B  P  SB  . 
 .  B  .  . 
 A  .  .  . 
-------------

AI decided to move right.
Wumpus World:
-------------
 G  SB  W  S 
 B  P  SB  . 
 .  B  .  . 
 .  A  .  . 
-------------

AI decided to move right.
Out of moves! Game Over.
